# 1 필요 라이브러리 불러오기

In [1]:
!pip install pytorch-forecasting
import torch
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.tuner import Tuner

from pytorch_forecasting.metrics import MultiHorizonMetric
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet

from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

from sklearn.preprocessing import LabelEncoder

from google.colab import drive
drive.mount('/content/drive')

import random
import os
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2 전처리된 자료 불러오기

In [3]:
train_df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Data/final/preprocessed_train_7.parquet')
test_df = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/Data/final/preprocessed_test_7.parquet")
train_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/final/train.csv')
sample_submission_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/final/sample_submission.csv')

# 3 데이터셋 구성

In [4]:
train_df['month'] = train_df['month'].astype(str)
train_df['time_idx'] = train_df['time_idx'].astype(int)
train_df['sales_rate'] = train_df['sales_rate'].astype(float)
train_df['week_weekend'] = train_df['week_weekend'].astype(str)
train_df['special_day'] = train_df['special_day'].astype(str)

test_df['month'] = test_df['month'].astype(str)
test_df['time_idx'] = test_df['time_idx'].astype(int)
test_df['sales_rate'] = test_df['sales_rate'].astype(float)
test_df['week_weekend'] = test_df['week_weekend'].astype(str)
test_df['special_day'] = test_df['special_day'].astype(str)

max_prediction_length = 21
min_prediction_length = 21
max_encoder_length = 90
validation_duration = 0
training_cutoff = train_df["time_idx"].max() - (max_prediction_length + validation_duration)

mid_train_df_2 = train_df[train_df['time_idx'] <= (train_df["time_idx"].max() // 1.15)]
training_cutoff_2 = train_df["time_idx"].max() - (max_prediction_length + validation_duration)

training = TimeSeriesDataSet(
    train_df[lambda x: x['time_idx'] <= training_cutoff],
    time_idx="time_idx",
    target="sales_rate",
    group_ids=['product_nums'],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=min_prediction_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["major", "middle", 'sub', 'brand', 'shop'],
    static_reals=[],
    time_varying_known_categoricals=['month', 'week_weekend', 'special_day', 'day'],
    time_varying_known_reals=["keyword_cnt"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        'sales_rate', "average_month_sales_rate",'sales_rate_log','sales'],
    target_normalizer = GroupNormalizer(groups=["product_nums"], transformation = 'softplus', method="standard"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# synchronized는 끄면 psfa_1, 켜면 psfa_2
batch_size = 1605  # set this between 32 to 128
# validation = TimeSeriesDataSet.from_dataset(training, train_df, predict=True, stop_randomization=True)
validation = TimeSeriesDataSet.from_dataset(training, train_df, predict=True, stop_randomization=True)
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=12, batch_sampler = 'synchronized')
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=12, batch_sampler = 'synchronized')

training_2 = TimeSeriesDataSet(
    mid_train_df_2[lambda x: x['time_idx'] <= training_cutoff_2],
    time_idx="time_idx",
    target="sales_rate",
    group_ids=['product_nums'],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    min_prediction_length=min_prediction_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["major", "middle", 'sub', 'brand', 'shop'],
    static_reals=[],
    time_varying_known_categoricals=['month', 'week_weekend', 'special_day', 'day'],
    time_varying_known_reals=["keyword_cnt"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        'sales_rate', "average_month_sales_rate",'sales_rate_log','sales'],
    target_normalizer = GroupNormalizer(groups=["product_nums"], transformation = 'softplus', method="standard"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# synchronized는 끄면 psfa_1, 켜면 psfa_2
batch_size = 1605  # set this between 32 to 128
# validation = TimeSeriesDataSet.from_dataset(training, train_df, predict=True, stop_randomization=True)
validation_2 = TimeSeriesDataSet.from_dataset(training_2, mid_train_df_2, predict=True, stop_randomization=True)
train_dataloader_2 = training_2.to_dataloader(train=True, batch_size=batch_size, num_workers=12,  batch_sampler = 'synchronized')
val_dataloader_2 = validation_2.to_dataloader(train=False, batch_size=batch_size, num_workers=12,  batch_sampler = 'synchronized')

In [5]:
# 1. Load the study from study_path
dataset_name = 'dataset_4'

study_path = f"/content/drive/MyDrive/Colab Notebooks/Kkh/data/optuna_best_parameter/{dataset_name}/best_parameter_study.pkl"
with open(study_path, "rb") as f:
    study = pickle.load(f)

# 2. Get the best hyperparameters from the study
best_hyperparameters = study.best_trial.params

In [6]:
# class PSFA(MultiHorizonMetric):
#     def loss(self, y_pred, target):
#         y_pred = self.to_prediction(y_pred)
#         diff_value = torch.abs(target - y_pred)
#         max_value = torch.max(target, y_pred) + 1e-8
#         weight = target / (torch.sum(target, axis=1).view(y_pred.shape[0],1) + 1e-8)

#         loss = torch.sum((diff_value / max_value) * weight)

In [7]:
# psfa_1: synchronized 옵션 끈 상태
class PSFA_1(MultiHorizonMetric):
    def loss(self, y_pred, target):
        y_pred = self.to_prediction(y_pred)
        diff_value = torch.abs(target - y_pred)
        max_value = torch.max(target, y_pred) + 1e-8
        weight_denumerator = torch.sum(target, axis=1).view(y_pred.shape[0], 1) + 1e-8
        weight = target / weight_denumerator
        loss = ((diff_value / max_value) * weight) * (y_pred.shape[1])
        return loss

# psfa_2: synchronized 옵션 킨 상태
class PSFA_2(MultiHorizonMetric):
    def loss(self, y_pred, target):
        y_pred = self.to_prediction(y_pred)
        diff_value = torch.abs(target - y_pred)
        max_value = torch.max(target, y_pred) + 1e-8
        # 위까지가 (1589, 21)

        # 행을 더한다 = 같은일자의 1589개의 품목을 더한다 = (1, 21)이 나온다.
        weight_denumerator = torch.sum(target, axis=0).view(1, y_pred.shape[1]) + 1e-8
        # print(f"가중치 분모의 shape: {weight_denumerator.shape}") # (1, 21)이 나온다면 옳은 것.
        weight = target / weight_denumerator
        # print(f"가중치의 shape: {weight.shape}") # (1589, 21)이 아논다면 맞는 것.
        loss = ((diff_value / max_value) * weight) * (y_pred.shape[0])
        return loss

class SMAPE(MultiHorizonMetric):
    def loss(self, y_pred, target):
        y_pred = self.to_prediction(y_pred)
        print(f"1. y_pred의 shape {y_pred.shape}")
        loss =  (y_pred - target).abs() / ((y_pred.abs() + target.abs() + 1e-8)/2)
        print(f"2. loss shape {loss.shape}")
        return loss

In [10]:
# configure network and trainer
# best_model_path = '/content/drive/MyDrive/Colab Notebooks/Kkh/data/callback_dir/test-epoch=04-val_loss=2.89.ckpt'
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=False, mode="min")
checkpoint_callback = ModelCheckpoint(save_top_k=20, monitor = 'val_loss', mode = 'min', dirpath =f"/content/drive/MyDrive/Colab Notebooks/Kkh/data/callback_dir/final_submission", filename = "20230916-{epoch:02d}-{val_loss:.6f}-{train_loss_epoch:.6f}")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    gradient_clip_val=best_hyperparameters['gradient_clip_val'],
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback, checkpoint_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=best_hyperparameters['learning_rate'],
    hidden_size=best_hyperparameters['hidden_size'],
    attention_head_size=best_hyperparameters['attention_head_size'],
    dropout=best_hyperparameters['dropout'],
    hidden_continuous_size=best_hyperparameters['hidden_continuous_size'],
    # loss로 psfa_1,3을 주는지, psfa_2를 주는지 확인할 것.
    loss=PSFA_2(),
    optimizer="Ranger",
    reduce_on_plateau_patience=100,
    log_interval=5
)

print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Number of parameters in network: 607.6k


In [11]:
# configure network and trainer
early_stop_callback_2 = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=False, mode="min")
checkpoint_callback_2 = ModelCheckpoint(save_top_k=20, monitor = 'val_loss', mode = 'min', dirpath =f"/content/drive/MyDrive/Colab Notebooks/Kkh/data/callback_dir/final_submission", filename = "20230916-nopinfo-val_{epoch:02d}-{val_loss:.6f}-{train_loss_epoch:.6f}")
lr_logger_2 = LearningRateMonitor()  # log the learning rate
logger_2 = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer_2 = pl.Trainer(
    max_epochs=10,
    accelerator="gpu",
    gradient_clip_val=best_hyperparameters['gradient_clip_val'],
    #fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger_2, early_stop_callback_2, checkpoint_callback_2],
    logger=logger_2,
)

tft_2 = TemporalFusionTransformer.from_dataset(
    training_2,
    learning_rate=best_hyperparameters['learning_rate'],
    hidden_size=best_hyperparameters['hidden_size'],
    attention_head_size=best_hyperparameters['attention_head_size'],
    dropout=best_hyperparameters['dropout'],
    hidden_continuous_size=best_hyperparameters['hidden_continuous_size'],
    # loss로 psfa_1,3을 주는지, psfa_2를 주는지 확인할 것.
    loss=PSFA_2(),
    optimizer="Ranger",
    reduce_on_plateau_patience=100,
    log_interval=5
)

print(f"Number of parameters in network: {tft_2.size()/1e3:.1f}k")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Number of parameters in network: 607.6k


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

# 아래는 실행하지 못할 것이다. 코랩에선 gpu가 하나라 너무 오래걸린다.
# trainer_2.fit(
#     tft_2,
#     train_dataloaders=train_dataloader_2,
#     val_dataloaders=val_dataloader_2
# )

INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | PSFA_2                          | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 235 K 
3  | prescalers                     

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# model_path =  "/content/drive/MyDrive/Colab Notebooks/Kkh/data/callback_dir/train_test/dataset_4/psfa/230815-0-epoch=01-val_loss=0.007451.ckpt"
# # best_model_path = trainer.checkpoint_callback.best_model_path
# tft = TemporalFusionTransformer.load_from_checkpoint(model_path)